#### Taken from Global Nav

In [3]:
import time
import asyncio
from tdmclient import aw, ClientAsync
import math
import numpy as np

# conversion thymio speed to mm/s
Thymio_to_mms = 0.349
px_to_mm = 140/100
#Thymio_to_pxs = Thymio_to_mms * mm_to_px 

# Thymio connection
async def connect_Thymio():
    """
    Establish a connection with the Thymio if possible
    """
    global node, client
    try:
        client = ClientAsync()
        node = await asyncio.wait_for(client.wait_for_node(), timeout=2.0)
        await node.lock()
        print("Thymio connected")

    except asyncio.TimeoutError:
        print("Thymio not connected: Timeout while waiting for node.")
    except Exception as e:
        print(f"Thymio not connected: {str(e)}")
        
# Thymio disconnection
def disconnect_Thymio():
    """
    Enable to disconnect the Thymio
    """
    aw(node.stop())
    aw(node.unlock())
    print("Thymio disconnected")
        
# Thymio control motor speeds  
async def set_speeds(left_speed, right_speed):
    """
    Enable to set the speed of the Thymio's wheels
    """
    global node
    v = {
        "motor.left.target":  [left_speed],
        "motor.right.target": [right_speed],
    }
    await node.set_variables(v)
    
async def motors_stop():
    """
    Stop the Thymio
    """
    global node
    v = {
        "motor.left.target":  [0],
        "motor.right.target": [0],
    }
    await node.set_variables(v)    

# Turn a specified angle 

# Constants
ROTATION_SPEED = 100
TIME_FULL_TURN = (8800/1000)

async def turn(angle):
    # Calculate the time needed to turn through the required angle
    rotation_time = (abs(angle) / (2*np.pi)) * TIME_FULL_TURN

    # Turn robot on itself
    # Check the sign of angle
    if np.sign(angle) > 0:
        # If angle is positive, turn in one direction
        await set_speeds(ROTATION_SPEED, -ROTATION_SPEED)
    else:
        # If angle is negative, turn in the other direction
        await set_speeds(-ROTATION_SPEED, ROTATION_SPEED)

    # Wait required time
    time.sleep(rotation_time)

    # Stop the robot
    #await motors_stop()

# Constants
FORWARD_SPEED = 200  # Base speed
TIME_PER_MM = 15.5/1000  # Time it takes for the robot to travel one meter at base speed

async def move_forward(distance_px):
    # Calculate the time needed to travel the requested distance
    
    distance_mm = distance_px * px_to_mm
    travel_time = (distance_mm) * TIME_PER_MM
    
    # Robot moves forward
    await set_speeds(FORWARD_SPEED, FORWARD_SPEED)

    # Wait for the necessary time
    time.sleep(travel_time)
    #print("End of forward after : ", travel_time)

In [4]:
await connect_Thymio()

Thymio connected


In [5]:
disconnect_Thymio()

Thymio disconnected


# Local Navigation 

In [6]:
node = await client.wait_for_node()
await node.lock()

Node 7ceccd72-4650-4181-812c-177085571dc0

In [7]:
await node.wait_for_variables()
node.var

{'_fwversion': [14, 0],
 '_id': [-1654],
 '_imot': [0, 0],
 '_integrator': [0, 0],
 '_productId': [8],
 '_vbat': [973, 975],
 'acc': [0, 0, 21],
 'acc._tap': [0],
 'button.backward': [0],
 'button.center': [0],
 'button.forward': [0],
 'button.left': [0],
 'button.right': [0],
 'buttons._mean': [11431, 9804, 14804, 14716, 16098],
 'buttons._noise': [85, 78, 89, 92, 110],
 'buttons._raw': [11449, 9801, 14818, 14716, 16093],
 'event.args': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'event.source': [-1654],
 'leds.bottom.left': [0, 0, 0],
 'leds.bottom.right': [0, 0, 0],
 'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
 'leds.top': [0, 0, 0],
 'mic._mean': [255],
 'mic.intensity': [3],
 'mic.threshold': [0],
 'motor.left.pwm': [0],
 'motor.left.speed': [-12],
 'motor.left.target': [0],
 'motor.right.pwm': [0],
 'motor.right.speed': [0],
 'motor.right.target': [0],
 'prox.comm.rx'

In [8]:
# Connect to the Thymio robot
client.tdm_addr
client.tdm_port

8596

### Desciption
Local navigation involves adjusting the robot's path to navigate around unexpected obstacles in its immediate vicinity. This often requires the robot to deviate from the optimal path to execute avoidance maneuvers. Thymio has to navigate past random obstacles, not captured in the vision and not factored into the global path planning. 

Once the obstacle is successfully navigated, Thymio can either return to the optimal path or calculate a new one. 

Thymio is equipped with five front horizontal proximity sensors for it to detect unknown objects that emerge in its path. Our primary objective is to help Thymio skillfully navigate around these local obstacles whilst enabling ample time for subsequent planning of an optimal path toward the ultimate goal.

- **Input**

    - Horizontal proximity sensor values; constantly updated


- **Output**

    - Motion control command for robot translation and rotation


- **Challenges**

    - Determining the colour of our physical obstacles have to be white, to not have the camera capture the obstacle in global vision.
    - Determining the shape of our obstacles have to be cylindrical for an optimal maneuver around it.
    - Determining the fixed distance to move forward and theta (angle) for Thymio's rotation was challenging to set without prior knowledge of the obstacle's shape. If the distance was too short or the angle too small, the robot will run into the obstacle. This tok us multiple tries to reach the optimal `distance_forward` and `theta`, as it was challenging to determine when an obstacle is considered cleared. Thus we had to finetune our parameters multiple times to ensure the success of our Thymio's obstacle avoidance. 
    - Determining the obstacle threshold for when Thymio should initiate the local avoidance, `obstThrh`.

### Parameters 

| Name                | Purpose                                                                           | Units | Global?|
| :------------------- | :------------------------------------------------------------------------------- |-------|-----|
| `obstThrh`      | High obstacle threshold for switching from global navigation to local navigation state| Int   | Y |
| `angle_turned` | Total angle turned since initial direction, before local avoidance                     | Radians| Y |
| `theta` | Angle to turn back, for Thymio to be aligned with initial direction before local avoidance    | Radians | N |
| `distance_forward`| Amount of distance to move forward  | Meters | Y |
| `diff_lr`| Calculated difference between the most left and most right sensors | Int | N |


In [63]:
## Parameters for local navigation
pi = math.pi
angle_turned = 0
distance_forward = 1
obstThrh = 20     # high obstacle threshold to switch state from global nav to local

### Local Obstacle Avoidance

In [58]:
async def local_avoidance(sensor_prox, theta, dist):
    """
    1a. Obstacle detected right in front: Rotate only
    1b. Obstacle detected near sides: rotate + go forward
    2. Determine rotation left or right
    3. Wall following of the obstacle
    4. After obstacle clearance, turn back to initial direction
    """
    # Global variables
    global node, obstThrh
    
    # Calculate difference in Left & Right sensors
    diff_lr = sensor_prox[0] - sensor_prox[4]
    
    # Object detected right in front
    if sensor_prox[2] > obstThrh: 
        # Rotate only (to the left)
        await turn(pi/4)  
        theta += pi/4
        print("Front obstacle, turning right")
        await move_forward(100*dist)
        await turn(-theta/2)
        print("Turning back to initial direction")
        await move_forward(dist*100)
        await turn(-theta/2)
        await turn(theta/2)
        return
    else:
        # Obstacle threshold
        if max(sensor_prox) > obstThrh:
            if (sensor_prox[0] + sensor_prox[1]) > (sensor_prox[3] + sensor_prox[4]):
                await turn(pi/24)
                theta += pi/24
                await move_forward(dist*2)
                print("Left obstacle, turning right")
                if (diff_lr) > 0:
                    await turn(pi/30)
                    theta += pi/30
                    await move_forward(dist)
                    print("Left wall following")
                else:
                    await move_forward(dist)
                    print("Not wall following")
            elif (sensor_prox[3] + sensor_prox[4]) > (sensor_prox[0] + sensor_prox[1]):
                await turn(-pi/24)
                theta -= pi/24
                await move_forward(dist*2)
                print("Right obstacle, turning left")
                if (diff_lr) < 0:
                    await turn(-pi/30)
                    theta -= pi/30
                    await move_forward(dist)
                    print("Right wall following")
                else:
                    await move_forward(dist)
                    print("Not wall following")
            else:
                return  # Do nothing
    
    # Obstacle cleared, move forward for a while before turning back to initial direction & move forward slightly more
    await move_forward(110*dist)
    await turn(-theta*dist*3.5)
    print("Turning back to initial direction")
    await move_forward(140*dist)
    await turn(theta*dist*3)
    await move_forward(dist)
    
    await motors_stop()
    return

In [57]:
await connect_Thymio()

Thymio connected


In [59]:
## Test

# Read sensor values
await node.wait_for_variables()

for i in range(5):
# ^ change to this: 
# while True:
    sensor_prox = node["prox.horizontal"]
    print(list(sensor_prox))
    # Call obstacle avoidance function if object detected
    while sum(sensor_prox[i] > obstThrh for i in range(0, 5)) > 0: 
        await local_avoidance(sensor_prox, angle_turned, distance_forward) 
    await client.sleep(0.2)
print("Completed Test!")
await move_forward(10)
await motors_stop()

[2517, 2025, 0, 0, 0, 0, 0]
Left obstacle, turning right
Left wall following
Turning back to initial direction
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
Completed Test!


In [55]:
disconnect_Thymio()

Thymio disconnected
